In [2]:
import os
import string
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

from langchain_huggingface import HuggingFaceEmbeddings

In [18]:
tqdm.pandas()

# Loading dataset

In [7]:
# Diretório dos arquivos
diretorio = "data/full/sentences/"

dados = []

for nome_arquivo in os.listdir(diretorio):
    caminho = os.path.join(diretorio, nome_arquivo)
    if os.path.isfile(caminho):
        data = nome_arquivo.replace(".txt", "")  # remove extensão se houver
        with open(caminho, "r", encoding="utf-8") as f:
            for linha in f:
                sentenca = linha.strip()
                if sentenca:
                    dados.append({"data": data, "sentenca": sentenca})


df = pd.DataFrame(dados)
df["data"] = pd.to_datetime(df["data"], format="%d%m%Y", errors="coerce")
print(df.head())

        data                                           sentenca
0 2012-04-18  Essa pagina depende do javascript para abrir, ...
1 2012-04-18                           Relatórios de AIR e ARR.
2 2012-04-18                                     Atas do Copom.
3 2012-04-18        Atas do Comitê de Política Monetária-Copom.
4 2012-04-18                                    166ª Reunião.-.


In [15]:
df

,data,sentenca
0,2012-04-18,"Essa pagina depende do javascript para abrir, ..."
1,2012-04-18,Relatórios de AIR e ARR.
2,2012-04-18,Atas do Copom.
3,2012-04-18,Atas do Comitê de Política Monetária-Copom.
4,2012-04-18,166ª Reunião.-.
...,...,...
55930,2024-05-08,Saiba mais na nossa.
55931,2024-05-08,Gerenciar cookies.
55932,2024-05-08,Rejeitar cookies.
55933,2024-05-08,Aceitar cookies.


In [10]:
# Gerar embeddings usando modelo HuggingFace
embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

In [11]:
# Gera o vetor da palavra "inflação"
vetor_inflacao = embeddings.embed_query("inflação")

In [13]:
# Função para calcular a distância (cosine similarity invertida)
def calcular_distancia(sentenca, vetor_referencia):
    vetor_sentenca = embeddings.embed_query(sentenca)
    # Cosine similarity retorna valor entre -1 e 1; quanto mais próximo de 1, mais similar
    similaridade = cosine_similarity([vetor_sentenca], [vetor_referencia])[0][0]
    # Para transformar em "distância", usamos 1 - similaridade
    distancia = 1 - similaridade
    return distancia

In [19]:
# Aplica a função ao DataFrame
df["inflation_distance"] = df["sentenca"].progress_apply(lambda x: calcular_distancia(x, vetor_inflacao))

100%|██████████| 55935/55935 [6:02:04<00:00,  2.57it/s]   


In [1]:
df

NameError: name 'df' is not defined